In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# ensures that the graphs are displayed in the notebook along with the code
%matplotlib inline
plt.rcParams["figure.figsize"] = [20, 10]

# Import the ZOD DevKit
from zod.frames.zod_frames import ZodFrames

### Set path to dataset and choose version
version $\in$ {"mini", "full"}

In [ ]:
data_dir = "/staging/dataset_donation/round_2"
version = "mini"

### Initialize ZodFrames

In [ ]:
zod = ZodFrames(dataset_root=data_dir, version=version)

### Split into Training and Validation sets

In [ ]:
train = zod.get_split("train")
val = zod.get_split("val")
print("Training split: ", train)
print("Validation split: ", val)

# Data
#### This dataset includes data from 3 sensor modalities and calibrations for each sensor:  
1. **Camera** - DESCRIPTION OF CAMERAS?
2. **LiDAR** - DESCRIPTION OF LIDARS?
3. **OXTS** - DESCRIPTION OF OXTS

We will in the following cells go through how this data can be fetched

### Read calibration for all available sensors


In [ ]:
frame_id = "029229"
calib = zod.read_calibration(frame_id)
lidar_velodyne_calib = ["lidar_velodyne"]
camera_front_calib = calib.cameras["camera_front"]

### Visualize single image

In [ ]:
frame_id = "029229"
image_path = zod.get_image_path(frame_id)
image = plt.imread(image_path)

plt.axis("off")
plt.imshow(image)

### Visualize single LiDAR point cloud
 Lidar fields description:

| Name | Type | Units | Description |
| --- | --- | --- | --- |
| 'timestamp' | string |  seconds  | UTC timestamp of each point. |
| 'x' | double |  meters  | x coordinate of the point in lidar frame |
| 'y' | double |  meters  | y coordinate of the point in lidar frame |
| 'z' | double |  meters  | z coordinate of the point in lidar frame |
| 'intensity' | double |    | intensity level of each point in range [0..255] |
| 'diode_index' | integer |    | index of diode emitter which produced a point (1..128) |

In [ ]:
pcd = zod.read_pointcloud(frame_id, "lidar_velodyne", n_sweeps_before=10, n_sweeps_after=0, motion_compensation=True)
np.hstack((pcd.points, pcd.intensity[:, None])).shape

In [ ]:
# Visualize BEV
from zod.visualization.lidar_bev import BEVBox

object_annotations = zod.read_object_detection_annotation(frame_id)
box3d = object_annotations

bev = BEVBox()
bev_image = bev(
    np.hstack((pcd.points, pcd.intensity[:, None])),
    (
        np.array([obj.name for obj in object_annotations if obj.box3d]),
        np.concatenate([obj.box3d.center[None,:] for obj in object_annotations if obj.box3d], axis=0),
        np.concatenate([obj.box3d.size[None,:] for obj in object_annotations if obj.box3d], axis=0),
        [obj.box3d.orientation for obj in object_annotations if obj.box3d],
    ),
)

In [ ]:
# Visualize in image
from zod.visualization.lidar_on_image import visualize_lidar_on_image

lid_image = visualize_lidar_on_image(pcd, calib, image)
plt.axis("off")
plt.imshow(lid_image)

### Visualize OXTS

In [ ]:
from zod.visualization.oxts_visualization import plot_gps_track_from_dataset_sequence

frame_id = "029229"
oxts_data = zod.read_oxts(frame_id)

# plot GPS track on interactive map
plot_gps_track_from_dataset_sequence(oxts_data)

In [ ]:
from zod.visualization.oxts_on_image import visualize_gps_on_image

# visualize GPS track over image
timestamp = zod.get_timestamp(frame_id)
camera_calib = zod.read_calibration(frame_id).cameras["camera_front"]

gps_on_image = visualize_gps_on_image(oxts_data, timestamp, camera_calib, image)
gps_on_image = cv2.cvtColor(gps_on_image, cv2.COLOR_BGR2RGB)
plt.imshow(gps_on_image)
plt.title("GPS on image")
plt.show()

#### There are 4 types of annotationed objects:  
1. **dynamic_objects** - objects that can move (vehicles, pedestrians etc.) - annotated with 2D/3D bounding boxes
2. **static_objects** - non-movable objects (light poles, traffic signs etc.) - annotated with 2D/3D bounding boxes
3. **lane_markings** - lane markings and road paitings - annotated with polygons
4. **ego_road** (Doesn't exist for all frames) - polygons that shows the road where ego vehicle can drive - annotated with polygons 

### Object annotations
#### Visualize a single object

In [ ]:
from zod.visualization.object_visualization import overlay_object_2d_box_on_image

frame_id = "082291"
object_id = 31

object_annotations = zod.read_object_detection_annotation(frame_id)
box2d = object_annotations[object_id].box2d

image = plt.imread(zod.get_image_path(frame_id))
image = overlay_object_2d_box_on_image(image, box2d, color=(255, 0, 0), line_thickness=10)

plt.axis("off")
plt.imshow(image)

#### 3D Bounding boxes can also be visualized in image

In [ ]:
from zod.visualization.object_visualization import overlay_object_3d_box_on_image

frame_id = "018591"
object_ids = [61, 63, 66, 69, 77]

calib = zod.read_calibration(frame_id)
object_annotations = zod.read_object_detection_annotation(frame_id)

image = plt.imread(zod.get_image_path(frame_id))

for object_id in object_ids:
    image = overlay_object_3d_box_on_image(
        image, object_annotations[object_id].box3d, calib, color=(255, 0, 0), line_thickness=10
    )

plt.axis("off")
plt.imshow(image)

#### Visualize all objects (both static and dynamic) in a scene

In [ ]:
from zod.visualization.object_visualization import (
    overlay_object_2d_box_on_image,
    overlay_object_properties_on_image,
)

frame_id = "018591"

object_annotations = zod.read_object_detection_annotation(frame_id)

image = plt.imread(zod.get_image_path(frame_id))

text_areas = []
for object_index, object_annotation in enumerate(object_annotations):
    image = overlay_object_2d_box_on_image(image, object_annotation.box2d)
    image = overlay_object_properties_on_image(
        image, object_annotation, properties_list=["name"], color=(255, 255, 0), text_areas=[]
    )

plt.axis("off")
plt.imshow(image)

#### Visualize only vehicles

In [ ]:
from zod.visualization.object_visualization import (
    overlay_object_2d_box_on_image,
    overlay_object_properties_on_image,
)

frame_id = "018591"

object_annotations = zod.read_object_detection_annotation(frame_id)

image = plt.imread(zod.get_image_path(frame_id))

for object_index, object_annotation in enumerate(object_annotations):
    text_areas = []
    if object_annotation.name == "Vehicle":
        image = overlay_object_2d_box_on_image(image, object_annotation.box2d)
        image = overlay_object_properties_on_image(
            image,
            object_annotation,
            properties_list=["object_id"],
            color=(255, 255, 0),
            object_id=object_index,
            text_areas=[],
        )

plt.axis("off")
plt.imshow(image)

#### Visualize only pedestrians

In [ ]:
from zod.visualization.object_visualization import (
    overlay_object_2d_box_on_image,
    overlay_object_properties_on_image,
)

frame_id = "062592"

object_annotations = zod.read_object_detection_annotation(frame_id)

image = plt.imread(zod.get_image_path(frame_id))

for object_index, object_annotation in enumerate(object_annotations):
    text_areas = []
    if object_annotation.name == "Pedestrian":
        image = overlay_object_2d_box_on_image(image, object_annotation.box2d)
        image = overlay_object_properties_on_image(
            image,
            object_annotation,
            properties_list=["object_id"],
            color=(255, 255, 0),
            object_id=object_index,
            text_areas=[],
        )

plt.axis("off")
plt.imshow(image)

#### Visualize only pole objects

In [ ]:
from zod.visualization.object_visualization import (
    overlay_object_2d_box_on_image,
    overlay_object_properties_on_image,
)

frame_id = "009158"

object_annotations = zod.read_object_detection_annotation(frame_id)

image = plt.imread(zod.get_image_path(frame_id))

for object_index, object_annotation in enumerate(object_annotations):
    text_areas = []
    if object_annotation.name == "PoleObject":
        image = overlay_object_2d_box_on_image(image, object_annotation.box2d)
        image = overlay_object_properties_on_image(
            image,
            object_annotation,
            properties_list=["object_type"],
            color=(255, 0, 0),
            text_areas=[],
        )

plt.axis("off")
plt.imshow(image)

#### Visualize only Traffic Signs

In [ ]:
from zod.visualization.object_visualization import (
    overlay_object_2d_box_on_image,
    overlay_object_properties_on_image,
)

frame_id = "062592"

object_annotations = zod.read_object_detection_annotation(frame_id)

image = plt.imread(zod.get_image_path(frame_id))

for object_index, object_annotation in enumerate(object_annotations):
    text_areas = []
    if object_annotation.name == "TrafficSign":
        image = overlay_object_2d_box_on_image(image, object_annotation.box2d)
        image = overlay_object_properties_on_image(
            image,
            object_annotation,
            properties_list=["object_id"],
            color=(255, 0, 0),
            text_areas=[],
            object_id=object_index,
        )

plt.axis("off")
plt.imshow(image)

### Visualize lane marking annotations

In [ ]:
from zod.visualization.lane_markings_visualization import overlay_lane_markings_on_image

frame_id = "029229"
lane_markings_annotation = zod.read_lane_markings_annotation(frame_id)
image = plt.imread(zod.get_image_path(frame_id))
image = overlay_lane_markings_on_image(lane_markings_annotation, image)

plt.axis("off")
plt.imshow(image)

### Ego Road annotations

In [ ]:
from zod.visualization.ego_road_visualization import overlay_ego_road_on_image

frame_id = "062592"
ego_road_annotation = zod.read_ego_road_annotation(frame_id)
image = plt.imread(zod.get_image_path(frame_id))
image = overlay_ego_road_on_image(ego_road_annotation, image)

plt.axis("off")
plt.imshow(image)